# Home Work

В этой работе будем иследовать одну из популярнейших рекомендательных моделей - **Latent Factor Model** - https://arxiv.org/pdf/1912.04754. 

Перед выполнением задания нужно убедиться, что прогоняется бейзлайн. Для этого:
1) Скачайте  файлы - **node2name.json** и **clickstream.parque** с необходимыми данными
2) Положите в репозиторий ноутбука и запустите код

В этой работе вам нужно:
1) перебрать параметры модели - edim,batch_size, lr, epoch , num_negatives -   (по **1 балу - 5 балов**) 
2) Тип OPTIMIZER_NAME - (**4 бала за 5 оптимизаторов**)
3) На основе имеющихся данных собрать лучшую модель (по **precision@30**) и рассчитать ее метрики (**4 бала**)
4) Попробовать другие модели (например  als - https://benfred.github.io/implicit/ , gru4rec, sasrec  ) - за sasrec на хорошем уровне сразу **10 балов**. За другие модели по **3 бала**
5) По окончанию работы в mlflow настроить графики для сравнения моделей. Можно проявить фантазию, но обязательно должно быть сравнение с бейзлайном (данный ноутбук) против других моделей
6) В mlflow залогировать последнюю версию ноутбука - необходимое условия. Либо в github, но тогда прикрепить ссылку в [mlflow](http://84.201.128.89:90/) . Эксперимент в формате - **homework-\<name\>**
7) Доп балы (**20 баллов**) тому у кого будет наибольший скор на тесте. Но ваш ноутбук должен прогонятся и быть вопроизводимым.

Суммарно за работу **20 балов**

In [3]:
! pip list

Package                           Version
--------------------------------- ------------
aext_assistant                    0.4.0
aext_assistant_server             0.4.0
aext_core                         0.4.0
aext_core_server                  0.4.0
aext_shared                       0.4.0
aiobotocore                       2.7.0
aiofiles                          22.1.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
aiosqlite                         0.18.0
alabaster                         0.7.12
alembic                           1.13.1
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.5.1
anaconda-navigator                2.6.0
anaconda-project                  0.11.1
aniso8601                         9.0.1
anyio                             4.2.0
appdirs                  

In [3]:
import json

with open('data/node2name.json', 'r') as f:
    node2name = json.load(f)

node2name = {int(k):v for k,v in node2name.items()}


In [5]:
# node2name

In [7]:
import pandas as pd

df = pd.read_parquet('data/clickstream.parque')
df = df.head(100_000)

In [9]:
df['is_train'] = df['event_date']< df['event_date'].max() - pd.Timedelta('2 day')
df['names'] = df['node_id'].map(node2name)

In [11]:
df

,cookie_id,event_date,node_id,is_train,names
0,15157399,2024-02-21 11:20:01,1047840,True,root -> Транспорт -> Запчасти и аксессуары -> ...
1,15157399,2024-03-05 10:24:54,1047561,True,root -> Услуги -> Предложения услуг -> Красота...
2,15157399,2024-03-05 10:28:55,1047561,True,root -> Услуги -> Предложения услуг -> Красота...
3,15157399,2024-04-13 11:22:25,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...
4,15157399,2024-04-13 11:22:45,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...
...,...,...,...,...,...
99995,20324089388,2024-04-06 20:22:13,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99996,20324089388,2024-04-06 20:24:20,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99997,20324089388,2024-04-06 20:27:25,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."
99998,20324089388,2024-04-06 20:33:10,1114510,True,"root -> Личные вещи -> Одежда, обувь, аксессуа..."


In [13]:
train_cooks = df[df['is_train']]['cookie_id'].unique()
train_items = df[df['is_train']]['node_id'].unique()


df = df[(df['cookie_id'].isin(train_cooks)) & (df['node_id'].isin(train_items))]

In [15]:
user_indes, index2user_id = pd.factorize(df['cookie_id'])
df['user_index'] = user_indes

node_indes, index2node = pd.factorize(df['node_id'])
df['node_index'] = node_indes

/var/folders/xf/j3td0vwj0xqbz2j3tfnmww200000gn/T/ipykernel_78084/541588533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_index'] = user_indes
/var/folders/xf/j3td0vwj0xqbz2j3tfnmww200000gn/T/ipykernel_78084/541588533.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['node_index'] = node_indes


In [17]:
df['node_index'].max()

2175

In [19]:
df.head()

,cookie_id,event_date,node_id,is_train,names,user_index,node_index
0,15157399,2024-02-21 11:20:01,1047840,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,0
1,15157399,2024-03-05 10:24:54,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
2,15157399,2024-03-05 10:28:55,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
3,15157399,2024-04-13 11:22:25,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2
4,15157399,2024-04-13 11:22:45,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2


In [21]:
df_train, df_test = df[df['is_train']], df[~df['is_train']]
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


df_train.shape, df_test.shape

((96611, 7), (3333, 7))

In [23]:
# фиксируем все сиды для воспроизводимости

import os 
import random
import numpy as np 

DEFAULT_RANDOM_SEED = 123

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

In [25]:
import torch
from torch import nn
import random 
from tqdm.auto import tqdm

from torch.utils.data import Dataset, DataLoader


seedEverything()


class RecDataset(Dataset):
    def __init__(self, users, items, item_per_users):
        self.users = users
        self.items = items
        self.item_per_users=item_per_users

    def __len__(self):
        return len(self.users)

    def __getitem__(self, i):
        user = self.users[i]
        return torch.tensor(user), torch.tensor(self.items[i]), self.item_per_users[user]


class LatentFactorModel(nn.Module):
    def __init__(self, edim, user_indexes, node_indexes):
        super(LatentFactorModel, self).__init__()
        self.edim = edim
        self.users = nn.Embedding(max(user_indexes) + 1, edim)
        self.items = nn.Embedding(max(node_indexes) + 1, edim)

    def forward(self, users, items):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items(items)
        res = torch.einsum('be,bne->bn', user_embedings, item_embedings)
        return res 

    def pred_top_k(self, users, K=10):
        user_embedings = self.users(users).reshape(-1, self.edim )
        item_embedings = self.items.weight
        res = torch.einsum('ue,ie->ui', user_embedings, item_embedings)
        return torch.topk(res, K, dim=1)

    


def collate_fn(batch, num_negatives, num_items):
    users, target_items, users_negatives = [],[], []
    for triplets in batch:
        user, target_item, seen_item = triplets
        
        users.append(user)
        target_items.append(target_item)
        user_negatives = []
        
        while len(user_negatives)< num_negatives:
            candidate = random.randint(0, num_items)
            if candidate not in seen_item:
                user_negatives.append(candidate)
                
        users_negatives.append(user_negatives)
                
    
    positive = torch.ones(len(batch), 1)       
    negatives = torch.zeros(len(batch), num_negatives)
    labels = torch.hstack([positive, negatives])
    # print(torch.tensor(target_items))
    # print(users_negatives)
    items = torch.hstack([torch.tensor(target_items).reshape(-1, 1), torch.tensor(users_negatives)])
    return torch.hstack(users), items, labels

    

In [27]:
user2seen = df_train.groupby('user_index')['node_index'].agg(lambda x: list(set(x)))

In [29]:
seedEverything()

BATCH_SIZE = 50_000
NUM_NEGATIVES = 5
EDIM = 128
EPOCH = 10
OPTIMIZER_NAME = 'Adam'
LR = 1

train_dataset = RecDataset(df_train['user_index'].values, df_train['node_index'], user2seen)


dataloader = DataLoader(train_dataset, shuffle=True,num_workers=0, batch_size=BATCH_SIZE,collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))


model = LatentFactorModel(EDIM, user_indes, node_indes)
optimizer = torch.optim.Adam(model.parameters(), LR)
 
bar = tqdm(total = EPOCH )


  0%|          | 0/10 [00:00<?, ?it/s]

In [31]:
seedEverything()

for i in range(EPOCH):
    bar_loader = tqdm(total = len(dataloader) ,)
    losses = []
    for i in dataloader:
        users, items, labels = i
        optimizer.zero_grad()
        logits = model(users, items)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels
        )
        loss.backward()
        optimizer.step()
        bar_loader.update(1)
        bar_loader.set_description(f'batch loss - {loss.item()}')
        losses.append(loss.item())
    
    bar.update(1)
    bar.set_description(f'epoch loss - {sum(losses)/len(losses)}')
    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
USER = 0

preds = list(model.pred_top_k(torch.tensor([USER]), 10)[1][0].numpy())
df[(df['user_index'] == USER) & (df['node_index'].isin(user2seen[USER]))]['names'].tolist()


['root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Кузов',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения']

In [35]:
[node2name[index2node[i]] for i in preds]

['root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Система охлаждения',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Кузов',
 'root -> Услуги -> Предложения услуг -> Красота, здоровье -> СПА-услуги, массаж',
 'root -> Транспорт -> Автомобили -> С пробегом -> Volkswagen -> Golf',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Автосвет',
 'root -> Транспорт -> Автомобили -> Внедорожник',
 'root -> Услуги -> Предложения услуг -> Автосервис, аренда -> Автосервис -> Мойка и уход за авто',
 'root -> Транспорт -> Запчасти и аксессуары -> Запчасти -> Для автомобилей -> Топливная и выхлопная системы',
 'root -> Недвижимость -> Коммерческая недвижимость -> Продажа -> Помещение свободного назначения',
 'root -> Транспорт -> Запчасти и аксессуары -> Шины, диски и колёса -> Диски -> 19 -> Литые -> 5x -> 114.3']

In [37]:
K = 100

test_users = df_test['user_index'].unique()


preds = model.pred_top_k(torch.tensor(test_users), K)[1].numpy()
df_preds = pd.DataFrame({'node_index': list(preds),
                         'user_index': test_users,
                         'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

df_preds = df_preds.explode(['node_index', 'rank']).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left' ,
)
df_preds['relevant'] = df_preds['relevant'].fillna(0)

In [39]:
df_preds

,node_index,user_index,rank,relevant
0,11,1,0,1.0
1,8,1,1,0.0
2,10,1,2,0.0
3,7,1,3,0.0
4,4,1,4,0.0
...,...,...,...,...
66695,1299,1658,95,0.0
66696,794,1658,96,0.0
66697,207,1658,97,0.0
66698,1148,1658,98,0.0


In [41]:

def calc_hitrate(df_preds, K):
    return  df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].max().mean()

def calc_prec(df_preds, K):
    return  (df_preds[df_preds['rank']<K].groupby('user_index')['relevant'].mean()).mean()
    
hitrate = calc_hitrate(df_preds, K)

hitrate, K

(0.8170914542728636, 100)

In [43]:
prec = calc_prec(df_preds, 30)
prec

0.04302848575712144

In [45]:
df_train['node_index'].max()

2175

In [47]:
top_popular = df_train[['node_index']].assign(v=1).groupby('node_index').count().reset_index().sort_values(by='v').tail(K)['node_index'].values


In [49]:
node2name[index2node[top_popular[-1]]]

'root -> Транспорт -> Автомобили -> Внедорожник'

In [51]:
df_preds_top_poplular = pd.DataFrame({'node_index': [list(top_popular) for i in test_users], 'user_index': test_users, 'rank': [[j for j in range(0, K)]for i in range(len(test_users))]})


df_preds_top_poplular = df_preds_top_poplular.explode(
    ['node_index', 'rank']
).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left' ,
)
df_preds_top_poplular['relevant'] = df_preds_top_poplular['relevant'].fillna(0)

calc_hitrate(df_preds_top_poplular, K)

0.6836581709145427

In [53]:
calc_prec(df_preds_top_poplular, 30)

0.006796601699150424

In [94]:
# логируем бейзлайн

import mlflow

mlflow.set_tracking_uri('http://84.201.128.89:90/')
mlflow.set_experiment('homework-marakhimova')

model_name = 'LatentFactorModel_baseline'

with mlflow.start_run(run_name='baseline'):
    mlflow.log_metrics(
        {
            'hitrate': hitrate, 
            'prec_30': prec,
        }
    )
    mlflow.log_params(
        {
        'model_name': model_name,
        'edim': EDIM,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'epoch': EPOCH,
        'num_negatives': NUM_NEGATIVES,
        'optimizer_name': OPTIMIZER_NAME,
        }
    )

2024/05/26 19:46:38 INFO mlflow.tracking.fluent: Experiment with name 'homework-marakhimova' does not exist. Creating a new experiment.


# ДЗ

## Перебор параметров LFM
1. перебрать параметры модели - edim,batch_size, lr, epoch , num_negatives -   (по **1 балу - 5 балов**) 
2. Тип OPTIMIZER_NAME - (4 бала за 5 оптимизаторов)

In [170]:
import optuna

model_name = 'LatentFactorModel'


def objective(trial):
    # фиксируем сиды
    seedEverything()
    
    # параметры
    EDIM = trial.suggest_categorical('EDIM', [32, 64, 128, 256])
    BATCH_SIZE = trial.suggest_categorical('BATCH_SIZE', [2_000, 5_000, 10_000, 20_000, 25_000, 50_000])
    LR = trial.suggest_float('LR', 1e-3, 1, log=False)
    EPOCH = trial.suggest_int('EPOCH', 10, 30, step=5, log=False)
    NUM_NEGATIVES = trial.suggest_int('NUM_NEGATIVES', 5, 30, step=5, log=False)
    OPTIMIZER_NAME = trial.suggest_categorical('OPTIMIZER_NAME', ['Adam', 'SGD', 'RMSprop', 'Adagrad', 'Adamax'])
    
    
    train_dataset = RecDataset(df_train['user_index'].values,
                               df_train['node_index'],
                               user2seen)
    
    dataloader = DataLoader(train_dataset,
                            shuffle=True,
                            num_workers=0,
                            batch_size=BATCH_SIZE,
                            collate_fn=lambda x: collate_fn(x, NUM_NEGATIVES, max(df['node_index'].values)))


    model = LatentFactorModel(EDIM, user_indes, node_indes)
    
    if OPTIMIZER_NAME == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), LR)
    elif OPTIMIZER_NAME == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), LR)
    elif OPTIMIZER_NAME == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), LR)
    elif OPTIMIZER_NAME == 'Adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), LR)
    elif OPTIMIZER_NAME == 'Adamax':
        optimizer = torch.optim.Adamax(model.parameters(), LR)
        
    # обучение
    for i in range(EPOCH):
        losses = []
        for i in dataloader:
            users, items, labels = i
            optimizer.zero_grad()
            logits = model(users, items)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(
                logits, labels
            )
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            
    # тест
    K = 100
    test_users = df_test['user_index'].unique()


    preds = model.pred_top_k(torch.tensor(test_users), K)[1].numpy()
    df_preds = pd.DataFrame({'node_index': list(preds),
                             'user_index': test_users,
                             'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

    df_preds = df_preds.explode(['node_index', 'rank']).merge(
        df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
        on = ['user_index', 'node_index'],
        how='left' ,
    )
    df_preds['relevant'] = df_preds['relevant'].fillna(0)
    
    hitrate = calc_hitrate(df_preds, K)
    prec = calc_prec(df_preds, 30)
    
    # логируем в mlflow
    run_name = f'{model_name}_{trial.number}'
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_metrics(
            {
                'hitrate': hitrate, 
                'prec_30': prec,
            }
        )
        mlflow.log_params(
            {
            'model_name': model_name,
            'edim': EDIM,
            'batch_size': BATCH_SIZE,
            'lr': LR,
            'epoch': EPOCH,
            'num_negatives': NUM_NEGATIVES,
            'optimizer_name': OPTIMIZER_NAME,
            }
        )
        
    # оптимизируем по precision@30
    return prec


sampler = optuna.samplers.TPESampler(seed=DEFAULT_RANDOM_SEED)

study = optuna.create_study(study_name='LFM_tuning', direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=30)

[I 2024-05-26 21:48:51,067] A new study created in memory with name: LFM_tuning
[I 2024-05-26 21:49:39,193] Trial 0 finished with value: 0.038730634682658666 and parameters: {'EDIM': 32, 'BATCH_SIZE': 10000, 'LR': 0.34383483813471855, 'EPOCH': 25, 'NUM_NEGATIVES': 15, 'OPTIMIZER_NAME': 'RMSprop'}. Best is trial 0 with value: 0.038730634682658666.
[I 2024-05-26 21:50:19,552] Trial 1 finished with value: 0.042028985507246375 and parameters: {'EDIM': 256, 'BATCH_SIZE': 2000, 'LR': 0.2944203323424405, 'EPOCH': 25, 'NUM_NEGATIVES': 5, 'OPTIMIZER_NAME': 'RMSprop'}. Best is trial 1 with value: 0.042028985507246375.
[I 2024-05-26 21:52:00,085] Trial 2 finished with value: 0.046476761619190406 and parameters: {'EDIM': 128, 'BATCH_SIZE': 25000, 'LR': 0.48355122999844163, 'EPOCH': 30, 'NUM_NEGATIVES': 20, 'OPTIMIZER_NAME': 'RMSprop'}. Best is trial 2 with value: 0.046476761619190406.
[I 2024-05-26 21:52:34,886] Trial 3 finished with value: 0.04212893553223389 and parameters: {'EDIM': 256, 'BATCH_

In [172]:
results = study.trials_dataframe()

# посмотрим топ-10 триалов
sorted_results = results.sort_values(by='value', ascending=False)[results.columns[[0, 1, 5, 6, 7, 8, 9, 10]]]
sorted_results.head(10)

,number,value,params_BATCH_SIZE,params_EDIM,params_EPOCH,params_LR,params_NUM_NEGATIVES,params_OPTIMIZER_NAME
13,13,0.049775,20000,64,15,0.505301,15,Adagrad
17,17,0.049425,20000,32,20,0.968934,20,Adagrad
14,14,0.049225,20000,128,15,0.996651,15,Adagrad
28,28,0.049175,10000,128,20,0.219824,10,Adagrad
7,7,0.048976,25000,128,15,0.428919,15,Adagrad
5,5,0.048926,5000,64,25,0.513615,5,Adagrad
21,21,0.048576,20000,64,15,0.989269,15,Adagrad
23,23,0.048476,20000,32,20,0.910851,10,Adagrad
15,15,0.048326,20000,64,15,0.921498,15,Adagrad
27,27,0.048226,20000,256,15,0.632152,25,Adagrad


### Лучшая LFM модель
3. На основе имеющихся данных собрать лучшую модель (по precision@30) и рассчитать ее метрики (4 бала)

In [174]:
best_lfm_params = study.best_trial.params

seedEverything()

train_dataset = RecDataset(df_train['user_index'].values,
                           df_train['node_index'],
                           user2seen)
    
dataloader = DataLoader(train_dataset,
                        shuffle=True,
                        num_workers=0,
                        batch_size=best_lfm_params['BATCH_SIZE'],
                        collate_fn=lambda x: collate_fn(x, best_lfm_params['NUM_NEGATIVES'], max(df['node_index'].values)))


best_lfm_model = LatentFactorModel(best_lfm_params['EDIM'], user_indes, node_indes)

OPTIMIZER_NAME = best_lfm_params['OPTIMIZER_NAME']
LR = best_lfm_params['LR']

if OPTIMIZER_NAME == 'Adam':
    optimizer = torch.optim.Adam(best_lfm_model.parameters(), LR)
elif OPTIMIZER_NAME == 'SGD':
    optimizer = torch.optim.SGD(best_lfm_model.parameters(), LR)
elif OPTIMIZER_NAME == 'RMSprop':
    optimizer = torch.optim.RMSprop(best_lfm_model.parameters(), LR)
elif OPTIMIZER_NAME == 'Adagrad':
    optimizer = torch.optim.Adagrad(best_lfm_model.parameters(), LR)
elif OPTIMIZER_NAME == 'Adamax':
    optimizer = torch.optim.Adamax(best_lfm_model.parameters(), LR)

# обучение
for i in range(best_lfm_params['EPOCH']):
    losses = []
    for i in dataloader:
        users, items, labels = i
        optimizer.zero_grad()
        logits = best_lfm_model(users, items)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels
        )
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

# тест
K = 100
test_users = df_test['user_index'].unique()


preds = best_lfm_model.pred_top_k(torch.tensor(test_users), K)[1].numpy()
df_preds = pd.DataFrame({'node_index': list(preds),
                         'user_index': test_users,
                         'rank': [[j for j in range(0, K)]for i in range(len(preds))]})

df_preds = df_preds.explode(['node_index', 'rank']).merge(
    df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
    on = ['user_index', 'node_index'],
    how='left' ,
)
df_preds['relevant'] = df_preds['relevant'].fillna(0)

hitrate = calc_hitrate(df_preds, K)
prec = calc_prec(df_preds, 30)

# чекаем воспроизводимость
assert prec == study.best_trial.values[0], "model best quality differs from optuna"

print(f"Best {model_name} params: {best_lfm_params}",
      f"\t precision@30: {prec}",
      f"\t hitrate@100: {hitrate}", sep='\n')

Best LatentFactorModel params: {'EDIM': 64, 'BATCH_SIZE': 20000, 'LR': 0.5053005226688491, 'EPOCH': 15, 'NUM_NEGATIVES': 15, 'OPTIMIZER_NAME': 'Adagrad'}
	 precision@30: 0.049775112443778115
	 hitrate@100: 0.823088455772114


## Другие модели

4. Попробовать другие модели (например  als - https://benfred.github.io/implicit/ , gru4rec, sasrec  ) - за sasrec на хорошем уровне сразу **10 балов**. За другие модели по **3 бала**

In [178]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.6/761.6 kB 4.9 MB/s eta 0:00:00a 0:00:01


### ALS

#### Дефолтная модель

In [416]:
import implicit
from scipy.sparse import coo_matrix

USERS = df['user_index'].unique().tolist()
ITEMS = df['node_index'].unique().tolist()

rows = df_train['user_index'].values
cols = df_train['node_index'].values
als_train_data = np.ones(df_train.shape[0])

coo_train = coo_matrix((als_train_data, (rows, cols)), shape=(len(USERS), len(ITEMS)))
csr_train = coo_train.tocsr()

coo_train, csr_train

(<1664x2176 sparse matrix of type '<class 'numpy.float64'>'
 	with 96611 stored elements in COOrdinate format>,
 <1664x2176 sparse matrix of type '<class 'numpy.float64'>'
 	with 26181 stored elements in Compressed Sparse Row format>)

In [418]:
%%time
als_model = implicit.als.AlternatingLeastSquares(random_state=DEFAULT_RANDOM_SEED)
als_model.fit(csr_train)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 5.92 s, sys: 137 ms, total: 6.06 s
Wall time: 926 ms


In [420]:
ids, scores = als_model.recommend(test_users, csr_train[test_users], N=100, filter_already_liked_items=False)

als_preds = pd.DataFrame({'node_index': list(ids),
                      'user_index': test_users,
                      'rank': [[j for j in range(0, K)]for i in range(len(ids))]})

als_preds = als_preds.explode(['node_index', 'rank']).merge(
            df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
            on = ['user_index', 'node_index'],
            how='left' ,
)
als_preds['relevant'] = als_preds['relevant'].fillna(0)
als_preds

,node_index,user_index,rank,relevant
0,9,1,0,0.0
1,4,1,1,0.0
2,7,1,2,0.0
3,11,1,3,1.0
4,15,1,4,0.0
...,...,...,...,...
66695,517,1658,95,0.0
66696,919,1658,96,0.0
66697,485,1658,97,0.0
66698,264,1658,98,0.0


In [422]:
als_hitrate = calc_hitrate(als_preds, 100)
als_prec = calc_prec(als_preds, 30)

als_hitrate, als_prec

(0.7946026986506747, 0.04832583708145926)

#### Перебор параметров

- factors (int, optional) – The number of latent factors to compute
- regularization (float, optional) – The regularization factor to use
- alpha (float, optional) – The weight to give to positive examples.
- iterations (int, optional) – The number of ALS iterations to use when fitting data

In [432]:
model_name = 'ALS'


def als_objective(trial):
    # фиксируем сиды
    seedEverything()
    
    # параметры
    FACTORS = trial.suggest_int('FACTORS', 50, 500, step=10, log=False)
    REGULARIZATION = trial.suggest_float('REGULARIZATION', 1e-2, 1, log=False)
    ALPHA = trial.suggest_float('ALPHA', 1, 15, log=False)
    ITERATIONS = trial.suggest_int('ITERATIONS', 10, 30, step=5, log=False)
        
    # обучение
    als_model = implicit.als.AlternatingLeastSquares(random_state=DEFAULT_RANDOM_SEED,
                                                     factors=FACTORS,
                                                     regularization=REGULARIZATION,
                                                     alpha=ALPHA,
                                                     iterations=ITERATIONS)
    als_model.fit(csr_train, show_progress=False)
            
    # тест
    K = 100
    test_users = df_test['user_index'].unique()


    ids, scores = als_model.recommend(test_users,
                                      csr_train[test_users],
                                      N=100,
                                      filter_already_liked_items=False)

    als_preds = pd.DataFrame({'node_index': list(ids),
                              'user_index': test_users,
                              'rank': [[j for j in range(0, K)]for i in range(len(ids))]})

    als_preds = als_preds.explode(['node_index', 'rank']).merge(
                df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
                on = ['user_index', 'node_index'],
                how='left' ,
    )
    als_preds['relevant'] = als_preds['relevant'].fillna(0)
    
    hitrate = calc_hitrate(als_preds, K)
    prec = calc_prec(als_preds, 30)
    
    # логируем в mlflow
    run_name = f'{model_name}_{trial.number}'
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_metrics(
            {
                'hitrate': hitrate, 
                'prec_30': prec,
            }
        )
        mlflow.log_params(
            {
            'model_name': model_name,
            'factors': FACTORS,
            'regularization': REGULARIZATION,
            'alpha': ALPHA,
            'iterations': ITERATIONS
            }
        )
        
    # оптимизируем по precision@30
    return prec


sampler = optuna.samplers.TPESampler(seed=DEFAULT_RANDOM_SEED)

als_study = optuna.create_study(study_name='ALS_tuning', direction='maximize', sampler=sampler)
als_study.optimize(als_objective, n_trials=30)

[I 2024-05-26 23:47:36,989] A new study created in memory with name: ALS_tuning
[I 2024-05-26 23:47:41,580] Trial 0 finished with value: 0.046876561719140425 and parameters: {'FACTORS': 370, 'REGULARIZATION': 0.29327794160087567, 'ALPHA': 4.175920349898844, 'ITERATIONS': 20}. Best is trial 0 with value: 0.046876561719140425.
[I 2024-05-26 23:47:46,201] Trial 1 finished with value: 0.04707646176911544 and parameters: {'FACTORS': 380, 'REGULARIZATION': 0.42887539552321635, 'ALPHA': 14.730698777384617, 'ITERATIONS': 25}. Best is trial 1 with value: 0.04707646176911544.
[I 2024-05-26 23:47:49,783] Trial 2 finished with value: 0.047176411794102945 and parameters: {'FACTORS': 270, 'REGULARIZATION': 0.398196343012209, 'ALPHA': 5.804492226112171, 'ITERATIONS': 25}. Best is trial 2 with value: 0.047176411794102945.
[I 2024-05-26 23:47:52,929] Trial 3 finished with value: 0.04787606196901549 and parameters: {'FACTORS': 250, 'REGULARIZATION': 0.06908111764347266, 'ALPHA': 6.57261957462604, 'ITERA

In [434]:
als_results = als_study.trials_dataframe()

# посмотрим топ-10 триалов
sorted_als_results = als_results.sort_values(by='value', ascending=False)[als_results.columns[[0, 1, 5, 6, 7, 8]]]
sorted_als_results.head(10)

,number,value,params_ALPHA,params_FACTORS,params_ITERATIONS,params_REGULARIZATION
22,22,0.050475,11.047160,90,10,0.016965
16,16,0.050325,12.394207,120,10,0.012350
20,20,0.050075,10.476724,100,15,0.106246
25,25,0.050025,10.499559,90,15,0.126302
17,17,0.049925,11.826670,90,10,0.012165
21,21,0.049825,11.262267,110,15,0.098878
28,28,0.049725,7.326605,150,10,0.087517
19,19,0.049725,13.066451,180,10,0.281940
23,23,0.049525,13.538419,160,15,0.086144
10,10,0.049525,9.355292,80,10,0.020547


#### Лучшая ALS модель

In [480]:
best_als_params = als_study.best_trial.params
best_als_params = {k.lower(): v for k, v in best_als_params.items()}

seedEverything()

# обучение
best_als_model = implicit.als.AlternatingLeastSquares(**best_als_params,
                                                      random_state=DEFAULT_RANDOM_SEED)
best_als_model.fit(csr_train, show_progress=True)

# тест
K = 100
test_users = df_test['user_index'].unique()


ids, scores = best_als_model.recommend(test_users,
                                       csr_train[test_users],
                                       N=100,
                                       filter_already_liked_items=False)

als_preds = pd.DataFrame({'node_index': list(ids),
                          'user_index': test_users,
                          'rank': [[j for j in range(0, K)]for i in range(len(ids))]})

als_preds = als_preds.explode(['node_index', 'rank']).merge(
            df_test[['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
            on = ['user_index', 'node_index'],
            how='left' ,
)
als_preds['relevant'] = als_preds['relevant'].fillna(0)

hitrate = calc_hitrate(als_preds, K)
prec = calc_prec(als_preds, 30)

# чекаем воспроизводимость
assert prec == als_study.best_trial.values[0], "model best quality differs from optuna"

print(f"Best {model_name} params: {best_als_params}",
      f"\t precision@30: {prec}",
      f"\t hitrate@100: {hitrate}", sep='\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Best ALS params: {'factors': 90, 'regularization': 0.01696469037727071, 'alpha': 11.047160266367834, 'iterations': 10}
	 precision@30: 0.050474762618690654
	 hitrate@100: 0.8245877061469266


### SASRec

Не получилось норм качество(

In [55]:
# делаем последовательности взаимодействия с товарами для каждого юзера
sorted_train = df_train.sort_values(by=['user_index', 'event_date'])
sorted_test = df_test.sort_values(by=['user_index', 'event_date'])

sorted_train.head()

,cookie_id,event_date,node_id,is_train,names,user_index,node_index
0,15157399,2024-02-21 11:20:01,1047840,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,0
1,15157399,2024-03-05 10:24:54,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
2,15157399,2024-03-05 10:28:55,1047561,True,root -> Услуги -> Предложения услуг -> Красота...,0,1
3,15157399,2024-04-13 11:22:25,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2
4,15157399,2024-04-13 11:22:45,1047835,True,root -> Транспорт -> Запчасти и аксессуары -> ...,0,2


In [57]:
from collections import defaultdict

train_seqs = defaultdict(list)
for row in sorted_train.itertuples():
    train_seqs[row.user_index].append(row.node_index)
    
train_seqs[0]

[0, 1, 1, 2, 2, 2]

In [59]:
test_seqs = defaultdict(list)
for row in sorted_test.itertuples():
    test_seqs[row.user_index].append(row.node_index)
    
test_seqs[test_users[1]]

[56, 50, 50, 50, 50]

In [61]:
# ищем максимальную длину последовательности
max_len = max(df_train.groupby('user_index')['node_index'].count().max(), df_test.groupby('user_index')['node_index'].count().max())

# делаем паддинг до максимальной длины последовательности
def pad_seqs(seqs, max_len=max_len):
    padded_seqs = np.zeros((len(seqs), max_len), dtype=np.int32)
    
    for i, seq in enumerate(seqs):
        if len(seq) < max_len:
            padded_seqs[i, -len(seq):] = seq
        else:
            padded_seqs[i] = seq
    return torch.tensor(padded_seqs)

padded_train_seqs = pad_seqs(list(train_seqs.values()))
padded_test_seqs = pad_seqs(list(test_seqs.values()))

In [63]:
# создаем датасет
class SeqDataset(Dataset):
    def __init__(self, users, seqs, is_train=True):
        self.users = users
        self.seqs = seqs
        self.is_train = is_train

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        user = self.users[idx]
        seq = self.seqs[idx]
        if self.is_train:
            return user, seq[:-1], seq[1:]
        else:
            return user, seq[:-1]
    
train_dataset_sasrec = SeqDataset(list(train_seqs.keys()), padded_train_seqs)
test_dataset_sasrec = SeqDataset(list(test_seqs.keys()), padded_test_seqs, is_train=False)

In [65]:
# создаем даталоадеры
BATCH_SIZE = 32


train_loader_sasrec = DataLoader(train_dataset_sasrec, batch_size=BATCH_SIZE, shuffle=True)
test_loader_sasrec = DataLoader(test_dataset_sasrec, batch_size=BATCH_SIZE, shuffle=False)

In [116]:
# создаем модель
class SASRec(nn.Module):
    def __init__(self,
                 dim_emb: int,
                 num_heads: int,
                 num_layers: int,
                 num_nodes: int,
                 dropout: float,
                 max_len: int=max_len):
        """
        dim_emb: размерность эмбеддингов
        num_heads: кол-во голов в multihead attention
        num_layers: ко-во слоев трансформеров
        num_nodes: кол-во уникальных нод
        dropout: вероятность дропаута
        max_len: максимальная длина последовательности
        """
        super(SASRec, self).__init__()
        
        self.node_emb = nn.Embedding(num_nodes, dim_emb)
        self.pos_emb = nn.Embedding(max_len, dim_emb)
        
        enc_layer = nn.TransformerEncoderLayer(d_model=dim_emb,
                                               nhead=num_heads)
        self.trasformer_enc = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        
        self.dropout = nn.Dropout(p=dropout)
        
        self.fc = nn.Linear(dim_emb, num_nodes)

        
    def forward(self, x):
        seq_len = x.size(1)
        
        pos = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand_as(x)
        x = self.node_emb(x) + self.pos_emb(pos)
        
        self.hidden_states = self.trasformer_enc(x)
        
        x = self.dropout(self.hidden_states)
        
        logits = self.fc(x)
        return logits
    
    def recommend(self, x):
        self.forward(x)
        last_hidden_state = self.hidden_states[:, -1, :]
        
        logits = self.fc(last_hidden_state)
        return logits

In [120]:
DIM_EMB = 52
NUM_HEADS = 4
NUM_LAYERS = 3
NUM_NODES = df['node_index'].max() + 1
DROPOUT = 0.3

LR = 1e-3
NUM_EPOCHS = 10

seedEverything()

sasrec_model = SASRec(DIM_EMB, NUM_HEADS, NUM_LAYERS, NUM_NODES, DROPOUT)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(sasrec_model.parameters(), lr=LR)

bar = tqdm(total = NUM_EPOCHS)

/opt/anaconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/10 [00:00<?, ?it/s]

In [122]:
seedEverything()

for i in range(NUM_EPOCHS):
    bar_loader = tqdm(total = len(train_loader_sasrec) ,)
    losses = []
    for i in train_loader_sasrec:
        _, inputs, targets = i
        
        optimizer.zero_grad()
        
        outputs = sasrec_model(inputs)
        logits = outputs.view(outputs.shape[0] * outputs.shape[1], -1)
        labels = targets.view(-1).long()
    
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        bar_loader.update(1)
        bar_loader.set_description(f'batch loss - {loss.item()}')
        losses.append(loss.item())
    
    bar.update(1)
    bar.set_description(f'epoch loss - {sum(losses)/len(losses)}')

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

In [140]:
# предикт

sasrec_preds = pd.DataFrame()
for sasrec_users, sasrec_seqs in tqdm(test_loader_sasrec):
    sasrec_users = sasrec_users.numpy()
    
    outputs = sasrec_model(sasrec_seqs)
    recs = outputs[:, -1, :]
    
    #recs = sasrec_model.recommend(sasrec_seqs)
    recs = torch.topk(recs, 100, dim=1).indices.squeeze(0).cpu().numpy()
    
    part_preds = pd.DataFrame({'node_index': list(recs),
                              'user_index': sasrec_users,
                              'rank': [[j for j in range(0, K)]for i in range(len(recs))]})

    part_preds = part_preds.explode(['node_index', 'rank']).merge(
                df_test[df_test['user_index'].isin(sasrec_users)][['user_index', 'node_index']].assign(relevant=1).drop_duplicates(),
                on = ['user_index', 'node_index'],
                how='left' ,
    )
    part_preds['relevant'] = part_preds['relevant'].fillna(0)
    
    sasrec_preds = pd.concat([sasrec_preds, part_preds], ignore_index=True)
    
sasrec_preds.head()

  0%|          | 0/21 [00:00<?, ?it/s]

,node_index,user_index,rank,relevant
0,0,1,0,0.0
1,106,1,1,0.0
2,77,1,2,0.0
3,152,1,3,0.0
4,31,1,4,0.0


In [126]:
hitrate = calc_hitrate(sasrec_preds, K)
prec = calc_prec(sasrec_preds, 30)

hitrate, prec

(0.6821589205397302, 0.02093953023488256)